In [4]:
import os
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")
%cd .
%load_ext autoreload
%autoreload 2

/home/jh/code/til/til-23-cv


In [5]:
from ultralytics import YOLO
from PIL import Image
from time import sleep

model = YOLO("yolov5x6u.pt")

In [ ]:
result = model.predict(
    "http://farm5.staticflickr.com/4032/5134536454_184138d8b1_z.jpg",
    half=True,
    device="cuda:0",
)

In [ ]:
def imshow(arr):
    display(Image.fromarray(arr[..., ::-1]))


imshow(result[0].plot())

In [ ]:
model.clear_callback("on_train_batch_end")
model.train(cfg="cfg/yolov5x6u_mvp.yaml")

### Preview Augmentations

In [ ]:
def end_early(_):
    raise KeyboardInterrupt

model.add_callback("on_train_batch_end", end_early)
try:
    model.train(
        cfg="cfg/yolov5x6u_mvp.yaml",
        name="temp/temp",
        deterministic=True,
        workers=2,
        verbose=False,
    )
except KeyboardInterrupt:
    pass
sleep(1)
for im in list(model.trainer.plots.keys()):
    display(Image.open(im))